In [ ]:
from pegasus.params import estimator_utils
import tfds

model_dir = 'ckpt/pegasus_ckpt/'
model_params =  'fine_tune"
estimator = estimator_utils.create_estimator("", model_dir, True, 1000, 1,
                                                 model_params)


In [ ]:
VOCAB_CHKPT = 'ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model'
encoder = public_parsing_ops.create_text_encoder("sentencepiece", 
                                                     VOCAB_CHKPT)

def input_fn(params):
    file_paths = 'facebook_2015'
    f = open(f"{file_path}.txt","r")
    full_text = f.read()
    # Break into chunks of 1024 tokens
    # Fully aware that they tokenize on BPE Encoding, so chunking on whole words may results in a small amount of lost
    # tokens, which is fine to me.
    text = full_text.split()
    n = 1024
    inputs = [encoder.encode(' '.join(text[i:i+n])) for i in range(0,len(text),n)]
    #To generate new summaries, pass empty strings
    targets = [encoder.encoder('') for i in range(len(inputs))]
    
    
    
    input_dict = dict(
                  inputs=inputs,
                  targets=targets
                 )

    data = pd.DataFrame(input_dict)

    with tf.io.TFRecordWriter('evaluate_data.tfrecords') as writer:
        for row in data.values:
            inputs, targets = row[:-1], row[-1]
            example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs[0].encode('utf-8')])),
                    "targets": tf.train.Feature(bytes_list=tf.train.BytesList(value=[targets.encode('utf-8')])),
                }
            )
        )
        writer.write(example.SerializeToString())
    
    dataset = tf.data.TFRecordDataset(filenames = ['evaluate_data.tfrecords'])
    return ds
    
checkpoint_path = 'ckpt/pegasus_ckpt/fine_tune.model'
predictions = estimator.predict(
          input_fn=input_fn, checkpoint_path=checkpoint_path)
print(predictions)